In [1]:
from __future__ import division

import sys
import os
import warnings
warnings.filterwarnings('always', append=True)
import pprint
import time

import matplotlib.pyplot as plt
from matplotlib.widgets import Slider

In [2]:
import helpy
import helplt
import melting as melt

modules = [helpy, helplt, melt]

In [3]:
rcs = helplt.rcParam_diff('PREPYLAB')

                  KEY:     DEFAULT      =>     PREPYLAB    
           figure.dpi:      100.0       =>       72.0      
     figure.edgecolor:        w         =>   (1, 1, 1, 0)  
     figure.facecolor:        w         =>   (1, 1, 1, 0)  
       figure.figsize:    [6.4, 4.8]    =>    [6.0, 4.0]   
figure.subplot.bottom:       0.11       =>      0.125      
          text.usetex:        0         =>       ...       


In [4]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [5]:
rcs = helplt.rcParam_diff('POSTPYLAB', rcs)

                  KEY:     DEFAULT      =>     PREPYLAB     =>    POSTPYLAB    
           figure.dpi:      100.0       =>       72.0       =>       ...       
     figure.edgecolor:        w         =>   (1, 1, 1, 0)   =>       ...       
     figure.facecolor:        w         =>   (1, 1, 1, 0)   =>       ...       
       figure.figsize:    [6.4, 4.8]    =>    [6.0, 4.0]    =>       ...       
figure.subplot.bottom:       0.11       =>      0.125       =>       ...       
          interactive:        0         =>       ...        =>        1        
          text.usetex:        0         =>       ...        =>       ...       


In [6]:
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'invalid': 'warn', 'over': 'warn', 'under': 'ignore'}

In [62]:
reloaded = [str(m) for m in map(reload, modules) if m.__file__.endswith('.py')]
if reloaded:
    print '\n\t'.join(['reloaded:'] + reloaded)
    print

mess, commit = helpy.getcommit(True)
print mess, str(commit.authored_datetime)[:-9], commit.summary
assert commit.repo.active_branch.name in ('melt')

reloaded:
	<module 'melting' from 'melting.py'>

620faa4(melt+) 2017-08-07 22:23 WIP average merged melting sets


In [8]:
prefix_fmt = '/Users/leewalsh/Squares/melting/analysis/{particle}/{W}x_{arrange}/{W}x_{arrange}_{run}'

In [72]:
def load_melting_stuff(particle, W, arrange, run, **kwargs):
    prefix_fmt = '/Users/leewalsh/Squares/melting/analysis/{particle}/{W}x_{arrange}/{W}x_{arrange}_{run}'
    prefix = prefix_fmt.format(particle=particle, W=W, arrange=arrange, run=run)
    if prefix.endswith('_'):
        prefix = prefix[:-1]
    prefix = prefix.rstrip('_').replace('__', '_').replace('_/', '/')
    print 'loading', prefix
    sys.stdout.flush()

    meta = helpy.load_meta(prefix)
    data = helpy.load_data(prefix)
    mdata = np.load(prefix + '_MELT.npz')['data']

    if len(data) < len(mdata) or np.any(data['id'] != mdata['id']):
        tsets = helpy.load_tracksets(
            data, run_repair='interp', run_track_orient=True)
        # to get the benefits of tracksets (interpolation, stub filtering):
        data = np.concatenate(tsets.values())
        data.sort(order=['f', 't'])

    print '...',
    sys.stdout.flush()

    frames, mframes = helpy.load_framesets((data, mdata), ret_dict=False)
    end_index = np.searchsorted(mdata['f'], meta.get('end_frame', np.inf))
    shells = melt.split_shells(mdata[:end_index], zero_to=1, do_mean=kwargs.get('do_mean', True), maxshell=W//2)
    shell_means = melt.average_shells(shells, kwargs.get('stats', ['rad', 'dens', 'psi', 'phi']), 'f')
    plot_args = melt.make_plot_args(meta)
    print 'done'
    sys.stdout.flush()

    return prefix, meta, data, mdata, frames, mframes, shells, shell_means, plot_args

In [10]:
configs = helpy.transpose_dict_of_lists({
    'particle': ['diag', 'pdms', 'pdms', 'lego'],
    'W':        [12] + [11]*3,
    'arrange':  ['in']*2 + ['rand', ''],
    'runs':     [range(1, n+1) + ['MRG']
                 for n in (4, 5, 2, 2)],
})

config = configs[1]
run = 'MRG'
%lprun -f load_melting_stuff loaded = load_melting_stuff(run=run, **config)

loading /Users/leewalsh/Squares/melting/analysis/pdms/11x_in/11x_in_MRG
... done


# Shells vs. Time

In [63]:
save = True
stats = ['rad', 'dens', 'psi', 'phi'][1:]

for config in configs:
    ncols, nrows = len(stats), len(config['runs'])
    figsize = np.array(plt.rcParams['figure.figsize'])
    figsize *= [ncols, nrows]
    fig, axeses = plt.subplots(figsize=figsize,
                               ncols=ncols, nrows=nrows,
                               sharex='all', sharey='col')

    for run, axes in zip(config['runs'], axeses):
        loaded = load_melting_stuff(run=run, **config)
        prefix, meta, data, mdata, frames, mframes, shells, shell_means, plot_args = loaded

        for stat, ax in zip(stats, axes):
            ax = melt.plot_by_shell(
                shell_means, 'f', stat,
                start=meta['start_frame'],
                smooth=meta['fps'],
                ax=ax, **plot_args)

    fig.tight_layout(h_pad=0, w_pad=0)

    if save:
        save_name_pattern = '{prefix}_{stat}.pdf'
        save_name = save_name_pattern.format(prefix=prefix.rpartition('_')[0], stat='OP_shell_time')
        print 'saving to', save_name
        fig.savefig(save_name, bbox_inches='tight', pad_inches=0)
        plt.close(fig.number)

loading /Users/leewalsh/Squares/melting/analysis/diag/12x_in/12x_in_1
... done
loading /Users/leewalsh/Squares/melting/analysis/diag/12x_in/12x_in_2
... done
loading /Users/leewalsh/Squares/melting/analysis/diag/12x_in/12x_in_3
... done
loading /Users/leewalsh/Squares/melting/analysis/diag/12x_in/12x_in_4
... done
loading /Users/leewalsh/Squares/melting/analysis/diag/12x_in/12x_in_MRG
... done
saving to /Users/leewalsh/Squares/melting/analysis/diag/12x_in/12x_in_OP_shell_time.pdf
loading /Users/leewalsh/Squares/melting/analysis/pdms/11x_in/11x_in_1
... done
loading /Users/leewalsh/Squares/melting/analysis/pdms/11x_in/11x_in_2
... done
loading /Users/leewalsh/Squares/melting/analysis/pdms/11x_in/11x_in_3
... done
loading /Users/leewalsh/Squares/melting/analysis/pdms/11x_in/11x_in_4
... done
loading /Users/leewalsh/Squares/melting/analysis/pdms/11x_in/11x_in_5
... done
loading /Users/leewalsh/Squares/melting/analysis/pdms/11x_in/11x_in_MRG
... done
saving to /Users/leewalsh/Squares/melti

# Parametric Histograms

In [12]:
assert False
save = False
stats_x = ['rad', 'dens']
stats_y = ['dens', 'psi', 'phi']

time_delta = 100
time_final = 500

nrows, ncols = len(stats_y), len(stats_x)
figsize = np.array(plt.rcParams['figure.figsize'])
figsize *= [ncols, nrows]

for config in configs[1:2]:
    for run in config['runs'][-1:]:
        loaded = load_melting_stuff(run=run, **config)
        prefix, meta, data, mdata, frames, mframes, shells, shell_means, plot_args = loaded

        time_final = min(time_final or plot_args['xylim']['f'][-1],
                         (len(mframes) - meta['start_frame'])/meta['fps'])
        time_steps = np.arange(0, time_final + time_delta, time_delta)
        time_label = ['start'] + time_steps.tolist() + ['end']
        fs = time_steps * meta['fps'] + meta['start_frame']
        periods = np.split(mframes, fs.astype(int))
        for p, period in enumerate(periods):
            if len(period) == 0:
                continue
            period = np.concatenate(period)
            fig, axes = plt.subplots(figsize=figsize, nrows=nrows, ncols=ncols,
                                     sharex='col', sharey='row')
            fig.suptitle(prefix+'\n'+r'$t\,f=[{},{})$'.format(time_label[p], time_label[p+1]))
            for j, x in enumerate(stats_x):
                for i, y in enumerate(stats_y):
                    if x == y: continue
                    ax = axes[i, j]
                    melt.plot_parametric_hist(period, x, y, ax=ax, **plot_args)


AssertionError: 

In [64]:
save = True
stats_x = ['rad']#, 'dens']
stats_y = ['dens', 'psi', 'phi']

time_delta = 100
time_final = 1000


place = {
    'stats_x': 'fig',
    'stats_y': 'fig',
    'config':  'fig',
    'run':     'ax_i',
    'time':    'ax_j',
}

for config in configs:
    runs = config['runs']
    time_final = time_final # or plot_args['xylim']['f'][-1]
    time_steps = np.arange(0, time_final + time_delta, time_delta)
    time_label = ['start'] + time_steps.tolist() + ['end']
    time_label = time_label[1:-1]

    nrows, ncols = len(runs), len(time_steps) + 1
    figsize = np.array(plt.rcParams['figure.figsize'])
    figsize *= [ncols, nrows]
    figs = {(x, y):
            plt.subplots(figsize=figsize,
                         nrows=nrows, ncols=ncols,
                         sharex='col', sharey='row')
            for x in stats_x for y in stats_y}

    for i, run in enumerate(runs):
        loaded = load_melting_stuff(run=run, **config)
        prefix, meta, data, mdata, frames, mframes, shells, shell_means, plot_args = loaded

        fs = time_steps * meta['fps'] + meta['start_frame']
        periods = np.split(mframes, fs.astype(int))[1:-1]
        for p, period in enumerate(periods):
            if len(period) == 0:
                continue
            period = np.concatenate(period)
            for x in stats_x:
                for y in stats_y:
                    if x == y: continue
                    fig, axes = figs[x, y]
                    ax = axes[i, p]
                    melt.plot_parametric_hist(period, x, y, ax=ax, **plot_args)
                    ax.set_title(r'$t\,f=[{},{})$'.format(
                        time_label[p], time_label[p+1]))

    if save:
        for (x, y), (fig, axes) in figs.iteritems():
            for ax in axes[:, 1:].flat:
                ax.set_ylabel('')
            for ax in axes[:-1, :].flat:
                ax.set_xlabel('')
            for ax in axes[1:, :].flat:
                ax.set_title('')
            for run, ax in zip(runs, axes[:, 0]):
                title = '{particle} {W}x{W} {arrange} {run}\n'.format(run=run, **config)
                ax.set_ylabel(title + ax.get_ylabel())
            fig.tight_layout()
            prefix = prefix_fmt.format(run=run, **config)
            prefix = prefix.rstrip('_').replace('__', '_').replace('_/', '/').rpartition('_')[0]
            savename = prefix + '_{x}_{y}.png'.format(x=x, y=y)
            print 'saving fig', fig.number, 'to', savename
            fig.savefig(savename, bbox_inches='tight', pad_inches=0)
            plt.close(fig.number)

loading /Users/leewalsh/Squares/melting/analysis/diag/12x_in/12x_in_1
... done
1 0 1 Axes(0.125,0.749828;0.0545775x0.130172) rad dens
1 0 2 Axes(0.125,0.749828;0.0545775x0.130172) rad psi
1 0 3 Axes(0.125,0.749828;0.0545775x0.130172) rad phi
1 1 1 Axes(0.190493,0.749828;0.0545775x0.130172) rad dens
1 1 2 Axes(0.190493,0.749828;0.0545775x0.130172) rad psi
1 1 3 Axes(0.190493,0.749828;0.0545775x0.130172) rad phi
1 2 1 Axes(0.255986,0.749828;0.0545775x0.130172) rad dens
1 2 2 Axes(0.255986,0.749828;0.0545775x0.130172) rad psi
1 2 3 Axes(0.255986,0.749828;0.0545775x0.130172) rad phi
1 3 1 Axes(0.321479,0.749828;0.0545775x0.130172) rad dens
1 3 2 Axes(0.321479,0.749828;0.0545775x0.130172) rad psi
1 3 3 Axes(0.321479,0.749828;0.0545775x0.130172) rad phi
1 4 1 Axes(0.386972,0.749828;0.0545775x0.130172) rad dens
1 4 2 Axes(0.386972,0.749828;0.0545775x0.130172) rad psi
1 4 3 Axes(0.386972,0.749828;0.0545775x0.130172) rad phi
1 5 1 Axes(0.452465,0.749828;0.0545775x0.130172) rad dens
1 5 2 Axes(0

MRG 6 1 Axes(0.517958,0.125;0.0545775x0.130172) rad dens
MRG 6 2 Axes(0.517958,0.125;0.0545775x0.130172) rad psi
MRG 6 3 Axes(0.517958,0.125;0.0545775x0.130172) rad phi
MRG 7 1 Axes(0.583451,0.125;0.0545775x0.130172) rad dens
MRG 7 2 Axes(0.583451,0.125;0.0545775x0.130172) rad psi
MRG 7 3 Axes(0.583451,0.125;0.0545775x0.130172) rad phi
MRG 8 1 Axes(0.648944,0.125;0.0545775x0.130172) rad dens
MRG 8 2 Axes(0.648944,0.125;0.0545775x0.130172) rad psi
MRG 8 3 Axes(0.648944,0.125;0.0545775x0.130172) rad phi
MRG 9 1 Axes(0.714437,0.125;0.0545775x0.130172) rad dens
MRG 9 2 Axes(0.714437,0.125;0.0545775x0.130172) rad psi
MRG 9 3 Axes(0.714437,0.125;0.0545775x0.130172) rad phi
saving fig 1 to /Users/leewalsh/Squares/melting/analysis/diag/12x_in/12x_in_rad_dens.png
saving fig 2 to /Users/leewalsh/Squares/melting/analysis/diag/12x_in/12x_in_rad_psi.png
saving fig 3 to /Users/leewalsh/Squares/melting/analysis/diag/12x_in/12x_in_rad_phi.png
loading /Users/leewalsh/Squares/melting/analysis/pdms/11x_i

5 0 3 Axes(0.125,0.254429;0.0545775x0.107857) rad phi
5 1 1 Axes(0.190493,0.254429;0.0545775x0.107857) rad dens
5 1 2 Axes(0.190493,0.254429;0.0545775x0.107857) rad psi
5 1 3 Axes(0.190493,0.254429;0.0545775x0.107857) rad phi
5 2 1 Axes(0.255986,0.254429;0.0545775x0.107857) rad dens
5 2 2 Axes(0.255986,0.254429;0.0545775x0.107857) rad psi
5 2 3 Axes(0.255986,0.254429;0.0545775x0.107857) rad phi
5 3 1 Axes(0.321479,0.254429;0.0545775x0.107857) rad dens
5 3 2 Axes(0.321479,0.254429;0.0545775x0.107857) rad psi
5 3 3 Axes(0.321479,0.254429;0.0545775x0.107857) rad phi
5 4 1 Axes(0.386972,0.254429;0.0545775x0.107857) rad dens
5 4 2 Axes(0.386972,0.254429;0.0545775x0.107857) rad psi
5 4 3 Axes(0.386972,0.254429;0.0545775x0.107857) rad phi
5 5 1 Axes(0.452465,0.254429;0.0545775x0.107857) rad dens
5 5 2 Axes(0.452465,0.254429;0.0545775x0.107857) rad psi
5 5 3 Axes(0.452465,0.254429;0.0545775x0.107857) rad phi
5 6 1 Axes(0.517958,0.254429;0.0545775x0.107857) rad dens
5 6 2 Axes(0.517958,0.254429

1 1 1 Axes(0.190493,0.657941;0.0545775x0.222059) rad dens
1 1 2 Axes(0.190493,0.657941;0.0545775x0.222059) rad psi
1 1 3 Axes(0.190493,0.657941;0.0545775x0.222059) rad phi
1 2 1 Axes(0.255986,0.657941;0.0545775x0.222059) rad dens
1 2 2 Axes(0.255986,0.657941;0.0545775x0.222059) rad psi
1 2 3 Axes(0.255986,0.657941;0.0545775x0.222059) rad phi
1 3 1 Axes(0.321479,0.657941;0.0545775x0.222059) rad dens
1 3 2 Axes(0.321479,0.657941;0.0545775x0.222059) rad psi
1 3 3 Axes(0.321479,0.657941;0.0545775x0.222059) rad phi
1 4 1 Axes(0.386972,0.657941;0.0545775x0.222059) rad dens
1 4 2 Axes(0.386972,0.657941;0.0545775x0.222059) rad psi
1 4 3 Axes(0.386972,0.657941;0.0545775x0.222059) rad phi
1 5 1 Axes(0.452465,0.657941;0.0545775x0.222059) rad dens
1 5 2 Axes(0.452465,0.657941;0.0545775x0.222059) rad psi
1 5 3 Axes(0.452465,0.657941;0.0545775x0.222059) rad phi
1 6 1 Axes(0.517958,0.657941;0.0545775x0.222059) rad dens
1 6 2 Axes(0.517958,0.657941;0.0545775x0.222059) rad psi
1 6 3 Axes(0.517958,0.657

# Spatial Maps

In [202]:
save = True
stats = ['sh', 'phi', 'psi']
#stats = ['dens']

time_delta = 25

for config in configs[-1:]:
    runs = config['runs'][:-1]  # cannot do spatial plot of merged data.

    for i, run in enumerate(runs):
        loaded = load_melting_stuff(run=run, **config)
        prefix, meta, data, mdata, frames, mframes, shells, shell_means, plot_args = loaded
        
        time_final = plot_args['xylim']['f'][-1]
        time_steps = np.arange(0, time_final, time_delta)[:15]
        time_label = time_steps.tolist()

        nrows, ncols = len(stats), len(time_steps)
        figsize = 2*np.array([ncols, nrows])
        fig, axes = plt.subplots(figsize=figsize,
                                 nrows=nrows, ncols=ncols,
                                 # sharex='all', sharey='all',
                                )

        fs = time_steps * meta['fps'] + meta['start_frame']
                
        norm_opts = plot_args.pop('norm')
        for i, stat in enumerate(stats):
            norm = norm_opts[stat]
            if norm is None:
                norm = (0, 1)
            if isinstance(norm, list):
                norm = tuple(np.percentile(mdata[stat]*plot_args['unit'][stat], norm))
            if isinstance(norm, tuple):
                norm = matplotlib.colors.Normalize(*norm)
            for j, f in enumerate(fs.astype(int)):
                ax = axes[i, j]
                melt.plot_regions(frames[f], mframes[f], norm=norm,
                                  ax=ax, colors=stat, **plot_args)
                if i == j == 0:
                    xylim = np.stack([ax.get_xlim(), ax.get_ylim()], 1)
                    # just shift:
                    xylim -= xylim.mean(0)
                    # square:
                    xylim = xylim.mean(1)
                else:
                    xc, yc = frames[f]['xy'].mean(0)
                    ax.set_xlim(xc + xylim)
                    ax.set_ylim(yc + xylim)

                ax.set_xticks([])
                ax.set_yticks([])
                if i == nrows-1:
                    ax.set_xlabel('time = {}'.format(time_steps[j]))
                ax.set_aspect('equal', adjustable='box-forced')

                if j == 0:
                    ax.set_ylabel(stat)
        if save:
            for ax in axes[:, 1:].flat:
                ax.set_ylabel('')
            for ax in axes[:-1, :].flat:
                ax.set_xlabel('')
            for ax in axes[1:, :].flat:
                ax.set_title('')
            #for stat, ax in zip(stats, axes[:, 0]):
                #title = '{particle} {W}x{W} {arrange} {run}\n'.format(run=run, **config)
                #ax.set_ylabel(title + ax.get_ylabel())
            fig.tight_layout()
            prefix = prefix_fmt.format(run=run, **config)
            prefix = prefix.rstrip('_').replace('__', '_').replace('_/', '/')
            savename =  prefix + '_spatial.png'
            print 'saving to', savename
            fig.savefig(savename, bbox_inches='tight', pad_inches=0)
            plt.close(fig.number)

loading /Users/leewalsh/Squares/melting/analysis/lego/11x/11x_1
... done
saving to /Users/leewalsh/Squares/melting/analysis/lego/11x/11x_1_spatial.png
loading /Users/leewalsh/Squares/melting/analysis/lego/11x/11x_2


/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.
  


... done
saving to /Users/leewalsh/Squares/melting/analysis/lego/11x/11x_2_spatial.png


In [ ]:
# Failed attempt at flexibility for placement
assert False
save = False
configs = helpy.transpose_dict_of_lists({
    'particle': ['diag', 'pdms', 'pdms'],
    'W':        [12, 11, 11],
    'arrange':   ['in']*2 + ['rand'],
    'runs':     [range(1, 5) + ['MRG'], range(1, 6) + ['MRG'], range(1, 3) + ['MRG']],
})

time_delta = 100
time_final = 500

vals = {
    'stats_x': ['rad', 'dens'],
    'stats_y': ['dens', 'psi', 'phi'],
    'configs': configs[1:2],
    'runs':    [1, 'MRG'],
    'time': np.arange(0, time_final + time_delta, time_delta),
}

place = {
    'stats_x': 'fig',
    'stats_y': 'fig',
    'config':  'ax_i',
    'run':     'ax_i',
    'time':    'ax_j',
}
counts = {
    'stats_x': len(vals['stats_x']),
    'stats_y': len(vals['stats_y']),
    'config':  len(vals['configs']),
    'run':     len(vals['runs']),
    'time':    len(vals['time']) + 1,
}

#time_final = time_final or plot_args['xylim']['f'][-1]
time_label = ['start'] + vals['time'].tolist() + ['end']

nfigs = [vals[v] for v in vals if place[v] == 'fig']
nrows = [vals[v] for v in vals if place[v] == 'ax_i']
ncols = [vals[v] for v in vals if place[v] == 'ax_j']

nfigs = np.multiply.reduce([vals[v] for v in vals if place[v] == 'fig'])
nrows = np.multiply.reduce([vals[v] for v in vals if place[v] == 'ax_i'])
ncols = np.multiply.reduce([vals[v] for v in vals if place[v] == 'ax_j'])
figsize = np.array(plt.rcParams['figure.figsize'])
figsize *= [ncols, nrows]

figs = [plt.subplots(figsize=figsize,
                     nrows=nrows, ncols=ncols,
                     sharex='col', sharey='row')
        for i in xrange(nfigs)]

fig.suptitle(prefix+'\n'+r'$t\,f=[{},{})$'.format(time_label[p], time_label[p+1]))

inds = {'fig': 0, 'ax_i': 0, 'ax_j': 0}
for i, config in enumerate(vals['configs']):
    for run in vals['runs']:
        loaded = load_melting_stuff(run=run, **config)
        prefix, meta, data, mdata, frames, mframes, shells, shell_means, plot_args = loaded

        fs = vals['time'] * meta['fps'] + meta['start_frame']
        periods = np.split(mframes, fs.astype(int))
        for p, period in enumerate(periods):
            if len(period) == 0:
                continue
            period = np.concatenate(period)
            for j, x in enumerate(stats_x):
                for i, y in enumerate(stats_y):
                    if x == y: continue
                    ax = axes[i, j]
                    melt.plot_parametric_hist(period, x, y, ax=ax, **plot_args)
        inds[place['run']] += 1
    inds[place['config']] += 1